In [1]:
# Clustering Toronto Neighbourhoods into 4 distincs clusters (Central, Down-Town, East & West Toronto) and looking at the venues in Central Toronoto using Foursqaure APIs"

In [2]:
import requests # import & load  necessary libraries and packages
import lxml.html as lh
import pandas as pd

In [3]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'  # wikipedia URL tnat has the Toronto Postcodes and details

In [4]:
from bs4 import BeautifulSoup  # import Beautifulsoup # Get BeautifulSoup package
print('BeautifulSoap imported')

BeautifulSoap imported


In [5]:
# Using BS to scrap the wikipedia HTML and generate a csv file with the postal codes etc
import requests  # import requests library

In [6]:
website_URL=requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

In [7]:
Toronto_soup = BeautifulSoup(website_URL,'lxml')  # Align it to Toronto

In [8]:
My_table = Toronto_soup.find('table',class_='wikitable sortable') # obtain wikitable sortable

In [9]:
#Create a handle, page, to handle the contents of the website
page = requests.get(url)

In [10]:
#Store the contents of the website under doc
doc = lh.fromstring(page.content)

In [11]:
#Parse data that are stored between <tr>..</tr> of HTML
tr_elements = doc.xpath('//tr')

In [12]:
#Check the length of the first 12 rows
    [len(T) for T in tr_elements[:12]]    

[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]

In [13]:
tr_elements = doc.xpath('//tr')
#Create empty list
Dict=[]
col=[]
i=0
#For each row, store each first element (header) and an empty list
for t in tr_elements[0]:
    i+=1
    name=t.text_content().strip("\n")
    print ('%d:%s'%(i,name))
    col.append((name,[]))

1:Postcode
2:Borough
3:Neighbourhood


In [14]:
#Since out first row is the header, data is stored on the second row onwards
for j in range(1,len(tr_elements)):
    #T is our j'th row
    T=tr_elements[j]
    
    #If row is not of size 10, the //tr data is not from our table 
    if len(T)!=3:
        break
    
    #i is the index of our column
    i=0
    
    #Iterate through each element of the row
    for t in T.iterchildren():
        data=t.text_content() 
        #Check if row is empty
        if i>0:
            try:
                data=int(data)
            except:
                pass
        #Append the data to the empty list of the i'th column
        col[i][1].append(data)
        #Increment i for the next column
        i+=1

In [15]:
[len(C) for (title,C) in col]

[288, 288, 288]

In [16]:
Dict={title:column for (title,column) in col}   # Dictionary with Postcodes, Borough and Neighbourhoods
df=pd.DataFrame(Dict)  # Creating the pandas Dataframe using the dictionary values

In [18]:
#df

In [20]:
df_Borough=df[df.Borough != 'Not assigned']   # get rid of "Not assigned" boroughs

In [23]:
#df_Borough

In [24]:
import numpy as np
df_Borough.columns = ['Postcode', 'Borough', 'Neighbourhood']

In [27]:
##df_Borough

In [28]:
df_Borough['Neighbourhood'] = df_Borough['Neighbourhood'].astype(str).str[:-1].astype(str)  # get rid of the eol marker from the neighbourhood values
#df_Borough['Neighbourhood']=df_Borough['Neighbourhood'].replace(to_replace=[r"\\t|\\n|\\r", "\t|\n|\r"], value=["",""], regex=True) 

/home/jupyterlab/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [30]:
#df_Borough

In [31]:
mask = df_Borough.Neighbourhood == 'Not assigned'  # assign Borough value to those neighbourhoods with "Not assigned"
column_name = 'Neighbourhood'
df_Borough.loc[mask, column_name] = df_Borough.Borough

/home/jupyterlab/conda/lib/python3.6/site-packages/pandas/core/indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
/home/jupyterlab/conda/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [33]:
#df_Borough

In [34]:

df_new=df_Borough.groupby('Postcode', sort=False)['Neighbourhood'].apply(','.join)  # group neighbourhoods by Postcode

In [36]:
#df_new

In [39]:

df_new_FINAL=pd.DataFrame({'Postcode':df_new.index, 'df_new':df_new.values})
df_new_F=df_new_FINAL.rename(columns={ df_new_FINAL.columns[1]: "Neighbourhood" })
df_N=df_new_F['Neighbourhood'].tolist() # create a neighbourhood list

In [41]:
#df_new

In [42]:

df_Borough_BONLY=df_Borough['Borough'] # create a Borough list
df_Borough_PONLY=df_Borough['Postcode'] # create a postcode list

In [46]:
#df_Borough_BONLY

In [45]:
#df_Borough_PONLY

In [47]:
df_Borough_BandP=pd.concat([df_Borough_PONLY, df_Borough_BONLY], axis=1)

In [49]:
#df_Borough_BandP

In [50]:
df_Borough_BandP_new = df_Borough_BandP.drop_duplicates(subset=['Postcode','Borough'])  # drop duplicates

In [52]:
#df_Borough_BandP_new

In [53]:
df_P=df_Borough_BandP_new['Postcode'].tolist() # refined lists
df_B=df_Borough_BandP_new['Borough'].tolist() # refined lists

In [56]:
#df_P
#df_B

In [57]:
Toronto_DICT= {'Postcode':df_P,'Borough':df_B, 'Neighbourhood':df_N} # Create a dictionary with the 3 refined lists
Toronto_df=pd.DataFrame(Toronto_DICT) # create the dataframe

In [58]:
# List of Postcodes, Boroughs, & Neighbourhoods in TORONTO FOR 103 POSTCODES
Toronto_df

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront,Regent Park"
3,M6A,North York,"Lawrence Heights,Lawrence Manor"
4,M7A,Queen's Park,Queen's Park
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Rouge,Malvern"
7,M3B,North York,Don Mills North
8,M4B,East York,"Woodbine Gardens,Parkview Hill"
9,M5B,Downtown Toronto,"Ryerson,Garden District"


In [60]:
##  SECOND SECTION

In [61]:
LandLfile = 'http://cocl.us/Geospatial_data'
Col_headers = ["Postcode","Latitude","Longitude"]  

In [62]:
df_Comp_Toronto = pd.read_csv(LandLfile,names=Col_headers,skiprows=1)  # getting the L& L data
print("Done")

Done


In [63]:
df_FINAL=pd.merge(Toronto_df, df_Comp_Toronto, on='Postcode', how='inner')  # merging the dataframes to get the L & L data annexed to the earlier DataFrame

In [65]:
df_FINAL

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights,Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
6,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
7,M3B,North York,Don Mills North,43.745906,-79.352188
8,M4B,East York,"Woodbine Gardens,Parkview Hill",43.706397,-79.309937
9,M5B,Downtown Toronto,"Ryerson,Garden District",43.657162,-79.378937


In [68]:
#THIRD SECTION (Clustering & Neighbourthood analysis.....)

In [70]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [71]:
df_Toronto_Special=df_FINAL[df_FINAL['Borough'].str.contains("Toronto")]   # select Boroughs with "Toronto"
df_Toronto_Special

,Postcode,Borough,Neighbourhood,Latitude,Longitude
2,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.654260,-79.360636
9,M5B,Downtown Toronto,"Ryerson,Garden District",43.657162,-79.378937
15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
19,M4E,East Toronto,The Beaches,43.676357,-79.293031
20,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
24,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
25,M6G,Downtown Toronto,Christie,43.669542,-79.422564
30,M5H,Downtown Toronto,"Adelaide,King,Richmond",43.650571,-79.384568
31,M6H,West Toronto,"Dovercourt Village,Dufferin",43.669005,-79.442259
36,M5J,Downtown Toronto,"Harbourfront East,Toronto Islands,Union Station",43.640816,-79.381752


In [74]:
address = 'Toronto, Ontario'
geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto City are 43.653963, -79.387207.


In [75]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)
#add markers to map
for lat, lng, borough, neighborhood in zip(df_Toronto_Special['Latitude'], df_Toronto_Special['Longitude'], df_Toronto_Special['Borough'], df_Toronto_Special['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color='blue',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(map_toronto)  
map_toronto

In [79]:
neighborhoods=df_Toronto_Special
CT_data = neighborhoods[neighborhoods['Borough'] == 'Central Toronto'].reset_index(drop=True)
CT_data.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790
1,M5N,Central Toronto,Roselawn,43.711695,-79.416936
2,M4P,Central Toronto,Davisville North,43.712751,-79.390197
3,M5P,Central Toronto,"Forest Hill North,Forest Hill West",43.696948,-79.411307
4,M4R,Central Toronto,North Toronto West,43.715383,-79.405678


In [80]:
address = 'Central Toronto, Ontario'  # selecting central Toronto (CT) for deep dive

In [82]:
    geolocator = Nominatim()
    location = geolocator.geocode(address)
    latitude = location.latitude
    longitude = location.longitude
    print('The geograpical coordinate of Central Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Central Toronto are 43.6449033, -79.3818364.


In [83]:
# create map of New York using latitude and longitude values
map_CT = folium.Map(location=[latitude, longitude], zoom_start=10)

In [84]:
# add markers to map
for lat, lng, borough, neighborhood in zip(CT_data['Latitude'], CT_data['Longitude'], CT_data['Borough'], CT_data['Neighbourhood']):
        label = '{}, {}'.format(neighborhood, borough)
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color='blue',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
          parse_html=False).add_to(map_CT) 
map_CT

In [85]:
CLIENT_ID = 'DO2ZJZCGCASHJIPQVJHVLYQ1LTOWASJSXQLPTGBTO4MY1JDG' # your Foursquare ID
CLIENT_SECRET = 'SN2VAVHL3QXMUIUOXPUMGXIJJA0Y0ZHRL5EXWNB4ICJUHWPP' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: DO2ZJZCGCASHJIPQVJHVLYQ1LTOWASJSXQLPTGBTO4MY1JDG
CLIENT_SECRET:SN2VAVHL3QXMUIUOXPUMGXIJJA0Y0ZHRL5EXWNB4ICJUHWPP


In [86]:
CT_data.loc[0, 'Neighbourhood']

'Lawrence Park'

In [87]:
neighborhood_latitude = CT_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = CT_data.loc[0, 'Longitude'] # neighborhood longitude value

In [88]:
neighborhood_name = CT_data.loc[0, 'Neighbourhood'] # neighborhood name

In [89]:
print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                                  neighborhood_latitude, 
                                                                  neighborhood_longitude))

Latitude and longitude values of Lawrence Park are 43.7280205, -79.3887901.


In [90]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius

In [91]:
# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
       CLIENT_ID,
        CLIENT_SECRET, 
        VERSION, 
        neighborhood_latitude, 
        neighborhood_longitude, 
        radius, 
        LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=DO2ZJZCGCASHJIPQVJHVLYQ1LTOWASJSXQLPTGBTO4MY1JDG&client_secret=SN2VAVHL3QXMUIUOXPUMGXIJJA0Y0ZHRL5EXWNB4ICJUHWPP&v=20180605&ll=43.7280205,-79.3887901&radius=500&limit=100'

In [92]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5d232ad2f129b50025e8a67e'},
 'response': {'warning': {'text': "There aren't a lot of results near you. Try something more general, reset your filters, or expand the search area."},
  'headerLocation': 'Toronto',
  'headerFullLocation': 'Toronto',
  'headerLocationGranularity': 'city',
  'totalResults': 3,
  'suggestedBounds': {'ne': {'lat': 43.7325205045, 'lng': -79.3825744605273},
   'sw': {'lat': 43.7235204955, 'lng': -79.3950057394727}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '50e6da19e4b0d8a78a0e9794',
       'name': 'Lawrence Park Ravine',
       'location': {'address': '3055 Yonge Street',
        'crossStreet': 'Lawrence Avenue East',
        'lat': 43.72696303913755,
        'lng': -79.39438246708775,
        'labeledL

In [93]:
# function that extracts the category of the venue
    def get_category_type(row):
        try:
            categories_list = row['categories']
        except:
            categories_list = row['venue.categories']
            
        if len(categories_list) == 0:
            return None
        else:
            return categories_list[0]['name']
        
    venues = results['response']['groups'][0]['items']
        
    nearby_venues = json_normalize(venues) # flatten JSON

In [94]:
# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng'] 
nearby_venues =nearby_venues.loc[:, filtered_columns]

In [95]:
# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

In [96]:
# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]
nearby_venues.head()

,name,categories,lat,lng
0,Lawrence Park Ravine,Park,43.726963,-79.394382
1,Zodiac Swim School,Swim School,43.728532,-79.382860
2,TTC Bus #162 - Lawrence-Donway,Bus Line,43.728026,-79.382805


In [97]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

3 venues were returned by Foursquare.
